In [3]:
import pde
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# SIRD model with diffusive mobility 
based on https://py-pde.readthedocs.io/en/latest/examples_gallery/pde_sir.html

In [10]:
SUSCEPTIBLE = "(S) Podatny"
INFECTIOUS = "(I) Zainfekowany"
RECOVERED = "(R) Wyleczony"
DECEASED = "(D) Zmarły"

class SIRD(pde.PDEBase):
    """SIRD model with diffusive mobility"""

    def __init__(self, beta=0.4, gamma=0.04, mi=0.005, diffusion=1.0, bc="auto_periodic_neumann"):
        """
            beta: transmission rate
            gamma: recovery rate
            mi: mortality
            diffusion: spatial mobility
            bc: boundary condition
        """
        super().__init__()
        self.beta = beta
        self.gamma = gamma
        self.mi = mi
        self.diffusion = diffusion
        self.bc = bc

    def get_initial_state(self, s: pde.ScalarField, i: pde.ScalarField)->pde.FieldCollection:
        """get initial state"""
        norm = (s + i).data.max()  # maximal density
        if norm > 1:
            s /= norm
            i /= norm

        # create deceased and recovered field
        d = pde.ScalarField(s.grid, data=0, label=DECEASED)
        r = pde.ScalarField(s.grid, data=1 - s - i - d, label=RECOVERED)
        return pde.FieldCollection([s, i, r, d])

    def evolution_rate(self, state, t=0):
        s, i, r, d = state
        ds_dt = self.diffusion * s.laplace(self.bc) - self.beta * i * s
        di_dt = self.diffusion * i.laplace(self.bc) + self.beta * i * s - self.gamma * i - self.mi * i
        dr_dt = self.diffusion * r.laplace(self.bc) + self.gamma * i
        dd_dt = d.laplace(self.bc) + self.mi * i
        return pde.FieldCollection([ds_dt, di_dt, dr_dt, dd_dt])

# Simulation plotting parameters

In [4]:
plot_2D_args={"vmin": 0, "vmax": 1,'cmap':'coolwarm'}
plot_1D_args={'ax_style': {'xlim': (0, 150), 'ylim': (0, 1)}}

# SIR 1D

In [13]:
equation = SIRD(beta=0.4, gamma=0.04, mi=0.005, diffusion=1)
# initialize state
grid = pde.UnitGrid([150])
s = pde.ScalarField(grid=grid, data=1, label=SUSCEPTIBLE)
i = pde.ScalarField(grid=grid, data=0, label=INFECTIOUS)
s.data[0] = 0
i.data[0] = 1
state = equation.get_initial_state(s, i)

storage = pde.MemoryStorage()
tracker = pde.PlotTracker(interval=10, plot_args=plot_1D_args)
solution = equation.solve(state, t_range=100, dt=1e-1, tracker=["progress", tracker, storage.tracker(interval=1)])

  0%|          | 0/100.0 [00:00<?, ?it/s]

Output()

Spent more time on handling trackers (3) than on the actual simulation (0.406)


In [15]:
#save simulation as movie
pde.movie(storage=storage, filename='./movies/pde_sird_1D.mp4', progress=True, show_time=True, plot_args=plot_1D_args)

# SIR 2D

In [15]:
equation = SIRD(beta=0.4, gamma=0.04, mi=0.005, diffusion=1)
# initialize state
grid = pde.UnitGrid([100, 100])
s = pde.ScalarField(grid=grid, data=1, label=SUSCEPTIBLE)
i = pde.ScalarField(grid=grid, data=0, label=INFECTIOUS)
s.data[0, 0] = 0
i.data[0, 0] = 1
state = equation.get_initial_state(s, i)

storage = pde.MemoryStorage()
tracker = pde.PlotTracker(interval=10, plot_args=plot_2D_args)
solution = equation.solve(state, t_range=100, dt=1e-1, tracker=["progress", tracker, storage.tracker(interval=1)])

  0%|          | 0/100.0 [00:00<?, ?it/s]

Output()

Spent more time on handling trackers (4.66) than on the actual simulation (1)


In [ ]:
#save simulation as movie
pde.movie(storage=storage, filename='./movies/pde_sird_2D.mp4', progress=True, show_time=True, plot_args=plot_1D_args)